### Shared Functions

In [1]:
def upper_nospace_string(s):
    # Remove all spaces
    s_no_spaces = s.replace(" ", "")
    # Convert all characters to uppercase
    s_upper = s_no_spaces.upper()
    return s_upper

### Prompt Engineering: Rule-based

In [2]:
import re


def process_query_rules(query):
    plate_number = None
    location = None

    # Define patterns to identify the type of query
    patterns = {
        "current_location": [
            r"where is vehicle ([A-Za-z0-9 ]+)?\??",
            r"what is the current location of vehicle ([A-Za-z0-9 ]+)?\??",
            r"location of vehicle ([A-Za-z0-9 ]+)?\??",
            r"current location of vehicle ([A-Za-z0-9 ]+)?\??",
            r"vehicle ([A-Za-z0-9 ]+) location\??",
        ],
        "dispatch_time": [
            r"what time was vehicle ([A-Za-z0-9 ]+)? dispatched (?:to|at|in|near) ([A-Za-z0-9 ]+)?\??",
            r"when was vehicle ([A-Za-z0-9 ]+)? dispatched (?:to|at|in|near) ([A-Za-z0-9 ]+)?\??",
            r"dispatch time of vehicle ([A-Za-z0-9 ]+)? (?:to|at|in|near) ([A-Za-z0-9 ]+)?\??",
            r"vehicle ([A-Za-z0-9 ]+)? dispatch time (?:to|at|in|near) ([A-Za-z0-9 ]+)?\??",
        ],
        "arrival_time": [
            r"what time did it arrive in the customer site\??",
            r"when did vehicle ([A-Za-z0-9 ]+)? arrive at the customer site\??",
            r"arrival time of vehicle ([A-Za-z0-9 ]+)? at customer site\??",
            r"vehicle ([A-Za-z0-9 ]+)? customer site arrival time\??",
            r"what time did vehicle ([A-Za-z0-9 ]+)? arrive (?:to|at|in|near) ([A-Za-z0-9 ]+)?\??",
            r"when did vehicle ([A-Za-z0-9 ]+)? arrive (?:to|at|in|near) ([A-Za-z0-9 ]+)?\??",
            r"arrival time of vehicle ([A-Za-z0-9 ]+)? (?:to|at|in|near) ([A-Za-z0-9 ]+)?\??",
            r"vehicle ([A-Za-z0-9 ]+)? (?:to|at|in|near) ([A-Za-z0-9 ]+) arrival time\??",
        ],
    }

    for query_type, pattern_list in patterns.items():
        for pattern in pattern_list:
            match = re.search(pattern, query.lower())
            if match:
                groups = match.groups()
                if len(groups) > 0 and groups[0]:
                    plate_number = upper_nospace_string(groups[0].strip())
                if len(groups) > 1 and groups[1]:
                    location = groups[1].strip().upper()

                return {
                    "plate_number": plate_number,
                    "location": location,
                    "query_type": query_type,
                }

    return {
        "plate_number": None,
        "location": None,
        "query_type": None,
    }

### Prompt Engineering: Use of LangChain

In [3]:
import os

from dotenv import load_dotenv
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# Initialize the language model
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", api_key=OPENAI_API_KEY)

# Define the prompt template
prompt_template = """
You are an intelligent assistant for a logistics company. Categorize the query and extract relevant information such as the plate number.
Plate number can be in the format ABC 1234 (3 letters followed by 4 numbers). Possible to not have space in between as well.
Query category can be:
1. current_location: asking where a vehicle is located
2. dispatch_time: asking about what time a vehicle left a specified location
3. arrival_time: asking about what time a vehicle arrived at a specific location
Location is a string such as warehouse A. It can also be an ID such as 21.

It is possible that the query asks about two vehicles or two locations. 
In those cases, return only the first instance identified.

Query: {query}

Format of Response:
Query Type: (current_location, dispatch_time, arrival_time, unknown)
Plate Number: (if not available, place None)
Location: (if not available, place None)
"""

prompt = PromptTemplate(input_variables=["query"], template=prompt_template)

# Define the chain
chain = LLMChain(prompt=prompt, llm=llm)


# Define the function to process the query using LangChain
def process_query_langchain(query):
    response = chain.run({"query": query})

    # Extract relevant details from the response
    query_type = None
    plate_number = None
    location = None

    # Parse the response
    if "Query Type:" in response:
        query_type_line = response.split("Query Type:")[1].split("\n")[0].strip()
        query_type = query_type_line.replace("(", "").replace(")", "").strip()

    if "Plate Number:" in response:
        plate_number_line = response.split("Plate Number:")[1].split("\n")[0].strip()
        if plate_number_line == "None":
            plate_number = None
        else:
            plate_number = upper_nospace_string(
                plate_number_line.replace("(", "").replace(")", "").strip()
            )

    if "Location:" in response:
        location_line = response.split("Location:")[1].split("\n")[0].strip()
        if location_line == "None":
            location = None
        else:
            location = location_line.replace("(", "").replace(")", "").strip().upper()

    return {
        "plate_number": plate_number,
        "location": location,
        "query_type": query_type,
    }

/Users/ryanabis/projects/ai-sampler/myenv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/Users/ryanabis/projects/ai-sampler/myenv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


### Retrieval-Augmented Generation (RAG): Retrieve info from Postgres

In [4]:
import os

import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

load_dotenv()

POSTGRES_URI = os.getenv("POSTGRES_URI")


def is_vehicle_registered(plate_number):
    engine = create_engine(POSTGRES_URI)
    query = text(
        """
        SELECT EXISTS (
            SELECT 1
            FROM vehicles
            WHERE UPPER(platenumber) = :plate_number
        )
    """
    )

    with engine.connect() as connection:
        result = connection.execute(query, {"plate_number": plate_number})
        exists = result.scalar()  # This will be True if the row exists, otherwise False

    return exists


def get_recent_vehicle_coordinates(plate_number):
    engine = create_engine(POSTGRES_URI)
    query = text(
        """
        SELECT *
        FROM movements
        WHERE UPPER(platenumber) = :plate_number
        ORDER BY createdAt desc
        limit 50
    """
    )

    df = pd.read_sql_query(query, engine, params={"plate_number": plate_number})
    engine.dispose()

    return df


def get_location_coordinates(location):
    engine = create_engine(POSTGRES_URI)
    query = text(
        """
        SELECT coordinates
        FROM locations
        WHERE UPPER(name) = :location
    """
    )

    df = pd.read_sql_query(query, engine, params={"location": location})
    engine.dispose()

    return df

In [5]:
def retrieve_db(processed_query):
    plate_number = processed_query["plate_number"]
    location = processed_query["location"]
    query_type = processed_query["query_type"]

    location_coordinates = None
    vehicle_coordinates = None

    is_registered = is_vehicle_registered(plate_number)
    if not is_registered:
        return None

    location_coordinates = get_location_coordinates(location)
    vehicle_coordinates = get_recent_vehicle_coordinates(plate_number)

    return {
        "vehicle_coordinates": vehicle_coordinates,
        "location_coordinates": location_coordinates,
    }

### Generating Response with GPT3/4

In [6]:
def generate_response(query):
    return query

### Difference in Prompt Engineering

In [7]:
# Example queries
queries = [
    "Where is vehicle ABC 123?",
    "What is the current location of vehicle ABC 1234?",
    # "What time was vehicle ABC 123 dispatched from Location 1?",
    # "What time did vehicle ABC123 arrive in Location 2?",
    # "Where is it location?",
]

for index, query in enumerate(queries):
    print(index)
    print("AI:", process_query_langchain(query))
    print("RULES:", process_query_rules(query))

0


/Users/ryanabis/projects/ai-sampler/myenv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


AI: {'plate_number': 'ABC123', 'location': None, 'query_type': 'current_location'}
RULES: {'plate_number': 'ABC123', 'location': None, 'query_type': 'current_location'}
1
AI: {'plate_number': 'ABC1234', 'location': None, 'query_type': 'current_location'}
RULES: {'plate_number': 'ABC1234', 'location': None, 'query_type': 'current_location'}


### End-to-end Run

In [8]:
# Example queries
queries = [
    "Where is vehicle ABC 123?",
    "What is the current location of vehicle ABC 1234?",
    # "What time was vehicle ABC 123 dispatched from Location 1?",
    # "What time did vehicle ABC123 arrive in Location 2?",
    # "Where is it location?",
]

for index, query in enumerate(queries):
    # processed_query = process_query_langchain(query)
    # relevant_db = retrieve_db(processed_query)
    response = generate_response(query)